In [5]:
! pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00


In [11]:
import pandas as pd

# Read the excel file into a pandas dataframe.
# Note that content/dataset (1).xlsx should be the actual path to your excel file.
df = pd.read_excel('/content/dataset (1).xlsx')

# Display the dataframe.
display(df)

,Id,Area,Bedrooms,Bathrooms,Floors,YearBuilt,Location,Condition,Garage,Price
0,1,1000,5,4,3,1970,Downtown,Excellent,No,1000000
1,2,2000,5,4,3,1958,Downtown,Excellent,No,900000
2,3,3000,2,2,3,1938,Downtown,Good,No,2200000
3,4,4000,4,2,2,1902,Suburban,Fair,Yes,200000
4,5,5000,1,4,2,1975,Downtown,Fair,Yes,300000
...,...,...,...,...,...,...,...,...,...,...
77,78,6000,3,3,1,1994,Rural,Good,Yes,9500000
78,79,5000,3,3,2,1977,Urban,Fair,No,1000000
79,80,4000,2,2,2,1975,Urban,Good,Yes,1200000
80,81,6000,3,3,3,1976,Rural,Good,Yes,630000


In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# ================================================
# 1. Load Data
# ================================================
df = pd.read_excel("dataset (1).xlsx")
df.columns = df.columns.str.strip().str.title()
df.drop_duplicates(inplace=True)
df.dropna(subset=["Bedrooms", "Bathrooms", "Floors", "Garage", "Condition", "Price"], inplace=True)
df["TotalRooms"] = df["Bedrooms"] + df["Bathrooms"]
df.reset_index(drop=True, inplace=True)

# ================================================
# 2. Model Training
# ================================================
features = ["Bedrooms", "Bathrooms", "Floors", "Garage", "Condition", "TotalRooms"]
X = pd.get_dummies(df[features], drop_first=True)
y = df["Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# ================================================
# 3. Prediction Function
# ================================================
def predict_price(bedrooms, bathrooms, floors, garage, condition):
    total_rooms = bedrooms + bathrooms
    input_data = pd.DataFrame([{
        "Bedrooms": bedrooms,
        "Bathrooms": bathrooms,
        "Floors": floors,
        "Garage": garage,
        "Condition": condition,
        "TotalRooms": total_rooms
    }])
    input_data = pd.get_dummies(input_data, drop_first=True)
    input_data = input_data.reindex(columns=X.columns, fill_value=0)
    prediction = model.predict(input_data)[0]
    return f"${prediction:,.2f}"

# ================================================
# 4. Gradio Interface
# ================================================
iface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Number(label="Bedrooms"),
        gr.Number(label="Bathrooms"),
        gr.Number(label="Floors"),
        gr.Number(label="Garage"),
        gr.Dropdown(["Excellent", "Good", "Fair"], label="Condition")
    ],
    outputs="text",
    title="House Price Predictor",
    description="Enter details to predict the price of a house."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6560adc2d0a157d356.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
